In [1]:
from rl import run_inference, ThorNavEnv, CLIPCuriosity
from models import ActorCritic
from cons import DEVICE
import torch
import prior
from ai2thor.controller import Controller

dataset = prior.load_dataset("procthor-10k")
train_scenes = dataset["train"]

NUM_UPDATES = 100

idx = torch.randint(0, len(train_scenes), (1,)).item()
idx = 3
house = train_scenes[idx]

controller = Controller(
    scene=house,
    snapToGrid=False,
    rotateStepDegrees=30,
    renderDepthImage=True,
)



/Users/viriyadhika/Documents/5.UofT/Courses/CSC2503H-ComputerVision/Project/NavAssistant/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching reference HEAD


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 24171.04it/s]


In [3]:

# Load trained model
actor_critic = ActorCritic().to(DEVICE)
actor_critic.load_state_dict(torch.load("data/ac_update_10.pt", map_location="cpu"))

clip_curiosity = CLIPCuriosity()
# Disable curiosity during inference
env = ThorNavEnv(controller, clip_curiosity=clip_curiosity)

# Run inference
traj, total = run_inference(actor_critic, env, max_steps=128, deterministic=False)

print("Episode return:", total)

Episode return: 2.873595075980774


In [5]:
for i in traj:
    print(i["logits"].softmax(dim=1))

tensor([[0.3343, 0.4031, 0.2627]])
tensor([[0.3374, 0.4426, 0.2200]])
tensor([[0.3365, 0.4790, 0.1845]])
tensor([[0.3325, 0.5061, 0.1614]])
tensor([[0.3271, 0.5267, 0.1462]])
tensor([[0.3234, 0.5403, 0.1363]])
tensor([[0.3193, 0.5508, 0.1299]])
tensor([[0.3164, 0.5581, 0.1255]])
tensor([[0.3142, 0.5632, 0.1226]])
tensor([[0.3109, 0.5679, 0.1212]])
tensor([[0.3093, 0.5698, 0.1209]])
tensor([[0.3090, 0.5715, 0.1195]])
tensor([[0.3097, 0.5715, 0.1188]])
tensor([[0.3117, 0.5728, 0.1155]])
tensor([[0.3139, 0.5704, 0.1157]])
tensor([[0.3123, 0.5724, 0.1153]])
tensor([[0.3102, 0.5739, 0.1160]])
tensor([[0.3123, 0.5667, 0.1210]])
tensor([[0.3120, 0.5631, 0.1248]])
tensor([[0.3106, 0.5617, 0.1278]])
tensor([[0.3153, 0.5555, 0.1292]])
tensor([[0.3179, 0.5525, 0.1296]])
tensor([[0.3193, 0.5512, 0.1295]])
tensor([[0.3201, 0.5508, 0.1292]])
tensor([[0.3172, 0.5573, 0.1254]])
tensor([[0.3153, 0.5616, 0.1231]])
tensor([[0.3152, 0.5627, 0.1221]])
tensor([[0.3123, 0.5670, 0.1207]])
tensor([[0.3114, 0.5

In [1]:
controller.stop()

NameError: name 'controller' is not defined